In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import KBinsDiscretizer

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.compose  import ColumnTransformer


In [7]:
df = pd.read_csv('titanic.csv', usecols=['Age','Fare', 'Survived'])

In [9]:
df.dropna(inplace=True)

In [11]:
df.shape

(714, 3)

In [13]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [15]:
X = df.iloc[:,1:]
y = df.iloc[:,0]



In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [19]:
X_train.head()

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542
253,30.0,16.1000
719,33.0,7.7750
666,25.0,13.0000


In [25]:
clf = DecisionTreeClassifier()

In [27]:
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [35]:
y_pred = clf.predict(X_test)

In [37]:
accuracy_score(y_test, y_pred)

0.6223776223776224

In [39]:
np.mean(cross_val_score(DecisionTreeClassifier(), X,y,cv=10, scoring='accuracy'))

0.6232981220657277

In [41]:
kbin_age = KBinsDiscretizer(n_bins=15, encode='ordinal', strategy='quantile')
kbin_fare = KBinsDiscretizer(n_bins=15, encode='ordinal', strategy='quantile')


In [43]:
trf = ColumnTransformer([
    ('first', kbin_age,[0]),
    ('second', kbin_fare,[1])
])

In [47]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [49]:
trf.named_transformers_['first'].bin_edges_

array([array([ 0.42,  6.  , 16.  , 19.  , 21.  , 23.  , 25.  , 28.  , 30.  ,
              32.  , 35.  , 38.  , 42.  , 47.  , 54.  , 80.  ])             ],
      dtype=object)

In [53]:
trf.named_transformers_['first'].bin_edges_

array([array([ 0.42,  6.  , 16.  , 19.  , 21.  , 23.  , 25.  , 28.  , 30.  ,
              32.  , 35.  , 38.  , 42.  , 47.  , 54.  , 80.  ])             ],
      dtype=object)

In [55]:
output = pd.DataFrame({
    'age':X_train['Age'],
    'age_trf':X_train_trf[:,0],
    'fare':X_train['Fare'],
    'fare_trf':X_train_trf[:,1]
})

In [61]:
output['age_labels'] = pd.cut(x=X_train['Age'],
                             bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['age_labels'] = pd.cut(x=X_train['Fare'],
                             bins=trf.named_transformers_['second'].bin_edges_[0].tolist())

In [65]:
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels
679,36.0,10.0,512.3292,14.0,"(108.9, 512.329]"
489,9.0,1.0,15.9000,7.0,"(14.454, 18.75]"
432,42.0,12.0,26.0000,9.0,"(18.75, 26.0]"
540,36.0,10.0,71.0000,12.0,"(51.479, 76.292]"
88,23.0,5.0,263.0000,14.0,"(108.9, 512.329]"


In [67]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2 = clf.predict(X_test_trf)

In [69]:
accuracy_score(y_test,y_pred2)

0.6363636363636364

In [73]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClasifier(), X,y,cv=10,scoring='accuracy'))

NameError: name 'DecisionTreeClasifier' is not defined